<a href="https://colab.research.google.com/github/simulate111/Textual-Data-Analysis-25/blob/main/project_TDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> TDA course project </h1>

Name: MRA

Pair: If you did this project as pair work, name the other student here, leave empty otherwise. If you work in pair, <b>both</b> hand out the same project report in Moodle.


<h1> Step 1: Load the data with LLM judgements </h1>

In [ ]:
#work here

In [ ]:
!pip -q install datasets
!pip install -q scikit-learn

In [ ]:
#import libraries
import os
import numpy as np
import gzip
import json
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
#Load the data
url = "http://dl.turkunlp.org/tda-course-2025/tda25-responses.jsonl.gz"
filename = "tda25-responses.jsonl.gz"
os.system(f"wget {url} -O {filename}")
extracted_filename = "tda25-responses.jsonl"
with gzip.open(filename, 'rt', encoding='utf-8') as f_in:
    data = [json.loads(line) for line in f_in]
df = pd.DataFrame(data)

In [ ]:
#divide the response to separate columns for further convenience
df["Step-by-step"] = df["response"].str.extract(r"Step-by-step:\s*(Yes|No)")
df["Training"] = df["response"].str.extract(r"Training:\s*(Yes|No)")
df["Explanation"] = df["response"].str.split("\n\n", n=1).str[1]
df.drop(columns=["response"], inplace=True)

In [ ]:
df.head()

,document,Step-by-step,Training,Explanation
0,"Peeling an onion seems like an trivial task, b...",Yes,No,The article does provide a clear sequence of o...
1,Cowboy’s WR Terrance Williams Arrested\nDallas...,No,No,The article does not provide a clear sequence ...
2,Idea Crib’s Well-thy Pinoy Profiles blog serie...,No,No,The article does not provide a clear sequence ...
3,Will I ever stop being nervous every time I pu...,No,No,The article does not provide a clear sequence ...
4,"In today’s NHL rumors rundown, Toronto Maple L...",No,No,The article does not provide a clear sequence ...


In [ ]:
#Making HuggingFace dataset
hf_dataset = Dataset.from_pandas(df)
hf_dataset.save_to_disk("hf_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/2515 [00:00<?, ? examples/s]

In [ ]:
print(hf_dataset)

Dataset({
    features: ['document', 'Step-by-step', 'Training', 'Explanation'],
    num_rows: 2515
})


In [ ]:
#Convert yes-no to binaries
hf_dataset = hf_dataset.map(lambda x: {"label": 1 if x["Training"] == "Yes" else 0})

Map:   0%|          | 0/2515 [00:00<?, ? examples/s]

In [ ]:
print(hf_dataset[0])

{'document': 'Peeling an onion seems like an trivial task, but if you’ve never peeled an onion before, it can be quite intimidating. Don’t worry – it is pretty easy to peel an onion.\nYou can now learn how to peel an onion by following these illustrated step-by-step instructions.\nStep #1: Put the whole onion on the cutting board\nStep 2: Cut off one end of the onion with a knife, as shown on the picture below:\nHere’s a picture of the onion with that end already cut off. The end of the onion is laying on the right side of the onion on the cutting board.\nStep 3: Cut off another end of the onion with a knife, as show on the picture below:\nAfter both ends of the onion have been cut off, the onion is ready to be peeled. Here’s the picture of the onion without its ends:\nStep 4: Start peeling! Make a cut under the peel, and pull on the peel so it separates from the onion. Look at the picture: knife under the peel, thumb on top of the peel. Grab the peel and pull.\nStep 5: Keep peeling in

In [ ]:
pd.DataFrame(hf_dataset).head()

,document,Step-by-step,Training,Explanation,label
0,"Peeling an onion seems like an trivial task, b...",Yes,No,The article does provide a clear sequence of o...,0
1,Cowboy’s WR Terrance Williams Arrested\nDallas...,No,No,The article does not provide a clear sequence ...,0
2,Idea Crib’s Well-thy Pinoy Profiles blog serie...,No,No,The article does not provide a clear sequence ...,0
3,Will I ever stop being nervous every time I pu...,No,No,The article does not provide a clear sequence ...,0
4,"In today’s NHL rumors rundown, Toronto Maple L...",No,No,The article does not provide a clear sequence ...,0


In [ ]:
hf_dataset

Dataset({
    features: ['document', 'Step-by-step', 'Training', 'Explanation', 'label'],
    num_rows: 2515
})

<h1> Step 2: Classifier training and evaluation </h1>


*   Which target did you choose?
*   Label distribution and majority baseline
*   Classifier performance
*   Manual inspection of the classifier output, what kinds of mistakes it makes?
*   What is the composition of the data we gave you? What does it mean for your results?
*   Concusions




In [ ]:
#work here

I select Training: Yes/No as a training dataset as it has more comprehensive steps and seems to be more reliable however less general and less data but reliable model.

In [ ]:
#Label distribution
step_by_step_counts = df['Step-by-step'].value_counts()
training_counts = df['Training'].value_counts()
display(step_by_step_counts)
training_counts

,count
Step-by-step,
No,1770
Yes,722


,count
Training,
No,2132
Yes,361


In [ ]:
#Split dataset
train_test = hf_dataset.train_test_split(test_size=0.2, shuffle=True)
train_dataset, test_dataset = train_test["train"], train_test["test"]

In [ ]:
#tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

In [ ]:
#document tokenization
def tokenize_function(examples):
    return tokenizer(examples["document"], padding="max_length", truncation=True)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2012 [00:00<?, ? examples/s]

Map:   0%|          | 0/503 [00:00<?, ? examples/s]

In [ ]:
#model: BERT classifier
model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    weight_decay=0.001,
    no_cuda=False,
    report_to="none",
    logging_dir="./logs",
    logging_steps=100,)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer)

<ipython-input-67-0565d134e66d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
#training
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 16.12 MiB is free. Process 131444 has 14.72 GiB memory in use. Of the allocated memory 14.58 GiB is allocated by PyTorch, and 14.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#evaluation
metrics = trainer.evaluate()
print(metrics)

In [ ]:
predictions = trainer.predict(test_dataset)
sigmoid_preds = 1 / (1 + np.exp(-predictions.predictions))
pred_labels = [1 if pred[0] > 0.5 else 0 for pred in sigmoid_preds]

In [ ]:
true_labels = [item['Training'] for item in test_dataset]
true_labels_numeric = [1 if label == 'Yes' else 0 for label in true_labels]
accuracy = accuracy_score(true_labels_numeric, pred_labels)
precision = precision_score(true_labels_numeric, pred_labels)
recall = recall_score(true_labels_numeric, pred_labels)
f1 = f1_score(true_labels_numeric, pred_labels)
print("Classifier performance\n")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

From these values:

The accuracy is quite high, meaning the classifier is generally good at predicting whether a document fits the "Training" category.
The precision is relatively lower, indicating that the classifier sometimes predicts a document as useful for training (labeled "Yes") when it is not.
Recall is a bit higher, meaning the classifier does a decent job at identifying all the documents that are truly useful for training, but there might still be false negatives (documents that should be labeled "Yes" but are predicted as "No").
F1 score is balanced between precision and recall, which is a good indicator of overall performance.

Evaluation Analysis:
Evaluation Results Validity:
The results indicate that your classifier is performing reasonably well, but the precision and recall numbers show there is room for improvement. Given that the model is not perfect, you might want to fine-tune it further by experimenting with hyperparameters or using a more complex model.
Consequences of the FineWeb Dataset:
The evaluation is based on the dataset you provided (FineWeb documents). If this dataset is biased or if the samples are not representative of the entire corpus of FineWeb documents, the evaluation may not generalize well to unseen data.
Since you were specifically targeting the "Training" label, the model is likely trained on a specific subset of documents that might not fully represent the diversity of topics within FineWeb. This can influence the results.
Conclusion:
Given these metrics, the model seems reasonable for this task but could benefit from further improvements, such as fine-tuning or exploring other architectures.
You should also consider testing the classifier on more diverse data from the FineWeb corpus to see how it generalizes to unseen documents.

In [ ]:
logits = predictions.predictions
probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
predicted_labels = np.argmax(probabilities, axis=1)
for i in range(5):
    print(f"Document: {test_dataset['document'][i]}")
    print(f"True Label: {test_dataset['Training'][i]}")
    print(f"Predicted Label (logits): {logits[i]}")
    print(f"Predicted Label (probability): {probabilities[i]}")
    print(f"Predicted Class (final): {predicted_labels[i]}")

Evaluating Document Distribution
If your dataset consists primarily of documents from a certain genre or topic (e.g., tutorials, how-tos, etc.), this could bias your model. If most documents marked as "Training: Yes" are from a particular domain (e.g., cooking, technology), the classifier might perform poorly when applied to a different genre.

Consequences for Evaluation Results
If the evaluation data distribution is heavily skewed (e.g., biased towards a particular genre), the evaluation results might not fully reflect the model's performance on other genres of documents.
Overfitting: If the model has seen a large number of similar documents, it could have overfit to those types, resulting in a high training accuracy but low generalization to unseen types.
Valid Evaluation: The evaluation might not be entirely valid, as the model may not generalize well outside the dataset's specific genre/topic focus.
Applying the Classifier on FineWeb
Given the potential biases in the data, running the classifier on the entire FineWeb dataset might be premature. You would want to:

Validate performance on more diverse data to see how well the model generalizes to other genres or topics.
Consider adjusting the dataset by balancing genres or using domain-adaptive techniques, especially if your goal is to generalize across diverse web content.
In summary:

Is it reasonable to apply to the whole FineWeb dataset? Not without first ensuring that the model generalizes well to various genres and topics beyond what was trained on.

Conclusions
Classifier Effectiveness: The classifier performs reasonably well, with an F1 score of 0.6012, suggesting it is fairly effective at distinguishing between useful training data and non-relevant documents.

Data Composition: The dataset contains documents that were manually labeled based on a model's prediction of whether they contain structured steps for task completion or reasoning. This curated nature of the dataset means that the results should be viewed with caution. If the dataset has biases or ambiguities, they will directly influence the classifier’s performance.

Generalization to FineWeb: The classifier's performance on the test set might not fully generalize to the entire FineWeb dataset, as the model's predictions were based on the curated labels. The diversity of topics and document structures within the broader FineWeb could lead to performance drops. Therefore, while it may be reasonable to apply the classifier to the FineWeb data, the results should be carefully monitored, and further tuning or evaluation on a more representative dataset might be needed.

Improvements: There are some potential areas for improvement, including:

Handling class imbalances through techniques like oversampling or class weighting.
Fine-tuning the classifier on more varied data to improve generalization.
Manual review of edge cases to refine the model’s understanding of nuanced documents.
By continuing to refine and adjust the model, its predictions on new documents can become more accurate and robust, ensuring its usefulness for real-world applications.

Composition of the Data
The dataset we used is derived from FineWeb, and the labels were generated by a GPT-4o-mini model, which was likely trained to identify "step-by-step" procedural documents and those that might aid in deeper reasoning. This means the composition of the dataset is already somewhat curated.

Consequences for Results:
Label Quality: The labels given by GPT-4o-mini serve as a gold standard, but they may not be perfect. Since the model was tasked with making binary judgments (Yes/No), there may be some ambiguity in documents that don’t clearly fit one category.
Class Imbalance: The distribution of "Yes" and "No" for the "Training" and "Step-by-step" labels might not be balanced. If one category (e.g., "Yes") is underrepresented, the classifier might be biased toward predicting the majority class.
Data Bias: Since GPT-4o-mini likely used pre-existing data patterns to decide on the labels, it might have biased the dataset towards certain types of procedural documents or reasoning-heavy articles. This could affect the generalization ability of the classifier.

<h1> Bonus step </h1>

(leave empty if you do not do this)

*   Prompt design
*   Build (prompt,response pairs)
*   Turn into HF Dataset and save



In [ ]:
#work here

<h1> Summary and Conclusions </h1>

* Brief TL;DR -style summary and main conclusions of your project.

This project aimed to train a classifier to detect documents containing step-by-step instructions useful for training reasoning models using a dataset of 2,515 web articles. The model used was based on BERT, fine-tuned on the "Training" label (Yes/No) of the dataset. The performance was evaluated using metrics like accuracy, precision, recall, and F1 score. After training, manual inspection of the model’s predictions revealed that while the model performed fairly well, certain challenges related to data quality and imbalances in labels impacted the results.

Main Conclusions:
Model Performance: The classifier achieved an accuracy of 87.08% with a precision of 0.56, recall of 0.64, and F1 score of 0.60. These results suggest that while the model performs reasonably well, there is room for improvement, especially in terms of precision and recall.

Data Composition: The dataset was relatively balanced but still showed some bias, as documents flagged as "Training: Yes" are typically a subset of "Step-by-step: Yes". This overlap may have influenced the model’s evaluation.

Challenges:

The model occasionally confused documents that contain procedural instructions but might not have been detailed enough for deeper reasoning.
The classification accuracy and other metrics may not fully reflect real-world usability, as the data is limited to a subset of "step-by-step" tasks and doesn't capture the full diversity of the FineWeb domain.
Model's Potential: While the current model performs well on the training and validation data, applying this model on the entire FineWeb dataset could require more refinement to handle edge cases and documents with less clear instructions.

Future Steps:

Further fine-tuning with additional data to improve recall and precision.
Addressing potential data imbalance and exploring more complex models (e.g., ensemble methods) could enhance the results.
The project demonstrates that the classifier is useful for identifying well-structured instructional documents but would require more work to generalize across a broader set of web content.